# **Apache Spark** 10k rows
- Autores: Anny Álvarez Nogales, Paula Arias Fernández, Jorge del Castillo Gómez

In [1]:
import numpy as np
from pyspark import SparkContext
sc = SparkContext("local[*]", "Practica1")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/26 21:00:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
def readFile(filename):
    text_file = sc.textFile(filename)
    text_file = text_file.map(lambda linea: np.array(linea.split(',')).astype(float))
    text_file= text_file.map(lambda x: (x[0:-1], int(x[-1])))
    return text_file    

CSV separado por comas ","

num filas= 1.000.000, num columnas= 12

Cada celda es un numero. 

La ultima celda de cada fila es la etiqueta (0/1)

FUNCIONES textFile, count, map, flatMap, reduce and reduceByKey. 

# **Ejercicio 1**

In [3]:
text_file= readFile('botnet_reduced_10k_l.csv')
text_file

PythonRDD[2] at RDD at PythonRDD.scala:53

In [4]:
def normalize (RDD_Xy):
    '''Normalize rdd'''
    rdd_rows = RDD_Xy.map(lambda row: row[0])  
    sumas = rdd_rows.reduce(lambda x, y: x + y)  
    means_array = sumas / RDD_Xy.count()
    
    rdd_diff = rdd_rows.map(lambda x_array: (x_array - means_array)**2)
    array_diff_sumatorio = rdd_diff.reduce(lambda x_row, y_row: x_row + y_row)
    array_varianza = np.maximum(array_diff_sumatorio / RDD_Xy.count(), 0)

    rdd_final = RDD_Xy.map(lambda row: (
        np.array([(value - means_array[idx_column]) / np.sqrt(array_varianza[idx_column]) 
                  for idx_column, value in enumerate(row[0])]), row[1]
    ) )
    return rdd_final

In [5]:
data = normalize(text_file)
data.take(2)

[(array([ 1.41281668, -0.75956071, -0.41204268, -0.45940748,  1.39580934,
         -0.35386218,  0.74057161, -0.8945781 , -0.40163083, -2.91039199,
          0.1567098 ]),
  0),
 (array([-0.79867306, -0.89146231,  3.64034447, -0.45940748, -0.52296869,
         -0.35387136,  0.55184122, -0.8945781 , -1.27444226,  0.4762026 ,
          0.1567098 ]),
  1)]

In [6]:
data_norm_test= data.map(lambda x : x[0])
data_norm_test.stdev()

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [7]:
def compute_sigmoid(z):
    y = (1 /(1 + np.exp(-z)))
    return y  


def predict(w, b, X):
    y = compute_sigmoid(np.dot(X,w) + b)
    if y >= 0.5:
        return 1
    else: 
        return 0


def accuracy(w,b, RDD_Xy):
    y_hat = RDD_Xy.map(lambda x: (predict(w, b, x[0]), x[1]))  
    preds = y_hat.map(lambda x: 1 if x[0] == x[1] else 0)
    correct_preds = preds.reduce(lambda x, y: x + y)    
    total = RDD_Xy.count()
    accuracy = correct_preds / total
    
    return accuracy


def train(RDD_Xy, iterations, learning_rate, lamba_reg):
    # inicializar pesos
    weights= np.random.rand(11)
    biases = np.random.rand(1)   

    for it in range(iterations):
        # número de datos
        m=RDD_Xy.count()
        # calcular predicción (aplicar sigmoide a la multiplicación de los pesos más el sesgo)
        rdd2=RDD_Xy.map(lambda x: (x[0], x[1],compute_sigmoid(np.dot(x[0],weights) + biases)))

        item_2_costFunction = (lamba_reg/(2*11))*np.array([w**2 for w in weights]).sum() 
        
        rdd2 = rdd2.map(lambda x: (x[0], x[1], x[2], -(x[1]*np.log(x[2]) + (1 - x[1])*np.log(1-x[2]))))
        coste=rdd2.map(lambda x:x[3] )
        coste=coste.reduce(lambda x,y:x+y )
        
        # gradientes
        rdd3= rdd2.map(lambda x: ((x[2]-x[1])*x[0],(x[2]-x[1])))
        rdd4= rdd3.reduce(lambda x,y: (x[0]+y[0], x[1]+y[1]))
        weights2 = (weights * lamba_reg)/ len(weights)
        d_weights= ((rdd4[0]/ m) + weights2 ) 
        d_bias= (rdd4[1]) / m
        # actualización de las derivadas
        weights = weights - learning_rate * d_weights
        biases = biases - learning_rate * d_bias


        coste=(coste/m) +  item_2_costFunction
        print(f"Iteration [{it}]: {coste}")
                
    return weights, biases

w, b, = train(data, 10, 1.5, 1)
acc = accuracy(w, b, data)
print ("Accuracy: ", acc)
    

Iteration [0]: [1.55015124]


Iteration [1]: [0.74621505]


Iteration [2]: [0.51013376]


Iteration [3]: [0.43356258]


Iteration [4]: [0.40450902]


Iteration [5]: [0.39189951]


Iteration [6]: [0.38591312]
Iteration [7]: [0.38289613]
Iteration [8]: [0.38130833]


Iteration [9]: [0.38044392]
Accuracy:  0.9263


# **Ejercicio 2**

In [8]:
import random
def transform(data, m, n_blocks):   
    # calcular n de elementos de cada bloque
    m_per_block = m // n_blocks

    # creamos una lista de índices que representan las filas del conjunto de datos. Tiene valores del 0 al m-1 (m es el número total de filas)
    indices = list(range(m)) 
    # mezclamos los índices para asegurarnos de que los datos se distribuyan de forma aleatoria entre los bloques
    random.shuffle(indices)

    # la operación modulo % asegura que el índice de cada fila se asigna a un bloque específico.
    # el resultado de 'indices.pop() % n_blocks' será un valor entre 0 y (n_blocks-1)
    # el operador hace que los datos se distribuyan entre los bloques de manera UNIFORME
    # los datos se asignen cíclicamente a bloques (0, 1, 2,... hasta n-1), y como esta desordenado ya se aplica el shuffle.

    # ahora se tiene el resultado de la operacion modulo (y se añade como clave a cada fila)
    indexed_data = data.map(lambda x: (indices.pop() % n_blocks, x)) 
    # 'x' es una fila del RDD original, y 'indices.pop() % n_blocks' le asigna un bloque aleatorio 

    return indexed_data.cache()


In [9]:
def get_block_data(data_cv, i, n_blocks):
    #agrupamos los datos por bloque usando el valor de la clave (n de bloque) y almacenamos los datos al lado
    #tengo (bloque/slice , dato (array y etiqueta))
    grouped_data = data_cv.map(lambda x: (x[0], [x[1]])) 

    
    #reduceByKey' para concatenar los elementos de la misma clave (bloque), es decir, agrupamos
    #los datos que tienen el mismo valor de bloque.
    grouped_data = grouped_data.reduceByKey(lambda a, b: a + b) 
    #aqui tenemos clave (n de bloque, lista de datos (filas con su array de 11 elementos, etiqueta))
    
    #en cada iteracion de este bucle trabajo con un slice o bloque 
    #tomamos todos los elementos cuyo bloque no sea el valor 'i' actual.
    tr_data = grouped_data.map(lambda x: (x[0], [item for item in x[1] if x[0] != i]))
    test_data = grouped_data.map(lambda x: (x[0], [item for item in x[1] if x[0] == i])) # el resto de test (elementos con clave el i)

    # hasta ahora tenia lista de listas para cada bloque (al agrupar con reducebykey)
    #aplano y tengo solo los datos no la clave del bloque (ya no se va a usar)
    tr_data = tr_data.flatMap(lambda x: x[1])  
    test_data = test_data.flatMap(lambda x: x[1])  
    
    return tr_data, test_data
    return tr_data, test_data

In [10]:
# read data
path = 'botnet_reduced_10k_l.csv'
data = readFile(path)

# standardize
data = normalize(data)

num_blocks_cv = 10
data_cv = transform(data, data.count(), num_blocks_cv)

nIter = 10
learningRate = 1.5
lamba_reg = 0

accuracies = []

for i in range(num_blocks_cv):
    tr_data, test_data = get_block_data(data_cv, i, num_blocks_cv)
    
    print(f"\nSlice {i+1}:")
    
    w,b = train(tr_data, nIter, learningRate, lamba_reg)
    acc = accuracy(w, b, test_data)
    print("acc :", acc)
    accuracies.append(acc)

avg_acc = sum(accuracies) / num_blocks_cv
print("average acc: ", avg_acc)
    


Slice 1:


Iteration [0]: [1.13970598]


Iteration [1]: [0.65566578]


Iteration [2]: [0.45528112]
Iteration [3]: [0.35905888]
Iteration [4]: [0.30800585]
Iteration [5]: [0.27786202]
Iteration [6]: [0.25825711]
Iteration [7]: [0.24453785]


Iteration [8]: [0.23440353]
Iteration [9]: [0.22660613]
acc : 0.9384615384615385

Slice 2:
Iteration [0]: [1.24227193]
Iteration [1]: [0.61869779]
Iteration [2]: [0.4150449]
Iteration [3]: [0.33560295]
Iteration [4]: [0.29493619]
Iteration [5]: [0.27019277]
Iteration [6]: [0.25344706]
Iteration [7]: [0.24129997]
Iteration [8]: [0.23205112]
Iteration [9]: [0.22475285]
acc : 0.9189723320158103

Slice 3:
Iteration [0]: [1.44634684]
Iteration [1]: [0.7302127]
Iteration [2]: [0.47138137]
Iteration [3]: [0.36914633]
Iteration [4]: [0.31752893]
Iteration [5]: [0.28650018]
Iteration [6]: [0.26574289]
Iteration [7]: [0.25087205]
Iteration [8]: [0.23969925]
Iteration [9]: [0.23100293]
acc : 0.9297872340425531

Slice 4:
Iteration [0]: [1.44295367]
Iteration [1]: [0.69371113]
Iteration [2]: [0.44479933]
Iteration [3]: [0.35326426]
Iteration [4]: [0.30702919]
Iteration [5]: [0.27896584]
Iteration [6]: [0.26004469]
Iteration [7]: [0.24639726]
Iteration [8]: [0.23607493]
Iteration [9]: [0.22798562]
a

Iteration [0]: [1.27860027]
Iteration [1]: [0.70275044]


Iteration [2]: [0.46568054]
Iteration [3]: [0.3578965]
Iteration [4]: [0.30294614]
Iteration [5]: [0.2713919]
Iteration [6]: [0.25132229]
Iteration [7]: [0.23753501]
Iteration [8]: [0.22750212]
Iteration [9]: [0.2198729]
acc : 0.9227799227799228

Slice 7:
Iteration [0]: [1.44701781]
Iteration [1]: [0.72455781]
Iteration [2]: [0.44506731]
Iteration [3]: [0.34057846]
Iteration [4]: [0.29131061]
Iteration [5]: [0.2636122]
Iteration [6]: [0.24606472]
Iteration [7]: [0.2339916]
Iteration [8]: [0.22517718]
Iteration [9]: [0.21844968]
acc : 0.926954732510288

Slice 8:
Iteration [0]: [1.08296151]
Iteration [1]: [0.59426097]
Iteration [2]: [0.40532188]
Iteration [3]: [0.32359708]
Iteration [4]: [0.28238287]
Iteration [5]: [0.25833751]
Iteration [6]: [0.24268693]
Iteration [7]: [0.23167744]
Iteration [8]: [0.22348443]
Iteration [9]: [0.21712573]
acc : 0.920997920997921

Slice 9:
Iteration [0]: [0.98099259]
Iteration [1]: [0.57986182]
Iteration [2]: [0.41886399]
Iteration [3]: [0.34484669]
Iterat